### Лабораторна работа 5.
Выполнили студенты группы **6404**
- Бахаев Дмитрий В.
- Сагалов Никита Д.

**5.1 Написать функцию формирования порождающей матрицы кода Рида-Маллера (r,m) в каноническом виде для произвольно заданных r и m.**

In [27]:
import numpy as np
from itertools import combinations, product

In [28]:
def reed_muller_generator_matrix(r, m):
    n = 2 ** m
    vectors = [format(i, f'0{m}b') for i in range(n)]
    matrix = []

    for k in range(r + 1):
        for I in combinations(range(m), k):
            row = []
            for v in vectors:
                value = 1
                for i in I:
                    value *= (int(v[i]) + 1) % 2
                row.append(value)
            matrix.append(row)
    
    return np.array(matrix)


In [29]:
print(reed_muller_generator_matrix(3, 4))

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]]


**5.2. Реализовать алгоритм мажоритарного декодирования для кода Рида-Маллера.**

In [30]:
def find_special(arr, n):
    unit_row = n
    for j in range(n):
        if arr[j] == 0:
            unit_row = j
            break
    j = unit_row
    steps = 0
    for j in range(unit_row, n, unit_row):
        if arr[j] == 0:
            steps += 1
        else:
            break
    x = unit_row*(steps+1)
    block = n
    for j in range(x, n, x):
        if arr[j] == 0:
            block = j
            break
    return unit_row, steps+1, block


def create_decode_array(r, m):
    n = 2**m
    E = [2 ** i for i in range(m - 1, -1, -1)]
    decode_array = []
    for k in range(r, 0, -1):
        arr = []
        for x in [sum(j) for j in list(combinations(E, m-k))]:
            v_t_array = np.zeros((2**(m-k), n), dtype=int)
            for j in range(n - 1, -1, -1):
                if x & j == x:
                    v_t_array[0][n - j - 1] = 1
            unit_row, steps, block = find_special(v_t_array[0], n)
            for i in range(1, len(v_t_array)):
                if i % steps == 0:
                    v_t_array[i, block:] = v_t_array[i-steps, :n-block]
                else:
                    v_t_array[i, unit_row:] = v_t_array[i-1, :n-unit_row]
            arr.append(v_t_array.T)
        decode_array.append(np.array(arr))
    decode_array.append([np.eye(n, dtype=int)])
    return decode_array


def decoding_word(w, decode_array, G, r, m):
    i = len(G) - 1
    word = np.zeros(i+1, dtype=int)
    w_ = w.copy()
    for v_t_array in decode_array:
        w_next = w_.copy()
        threshold_amount = len(v_t_array[0][0]) // 2
        for x in v_t_array:
            units_amount = (np.dot(w_, x) % 2).sum()
            if units_amount == threshold_amount:
                print(f'Допущена {2**(m-r-1)}-я ошибка. Сообщение не может быть восстановлено!')
                return None
            if units_amount > threshold_amount:
                w_next ^= G[i]
                word[i] = 1
            else:
                word[i] = 0
            i -= 1
        w_ = w_next
    return word

**5.3. Провести экспериментальную проверку алгоритма декодирования для кода Рида-Маллера (2,4).**

In [31]:
R = 2
M = 4
G = reed_muller_generator_matrix(R, M)
G

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]])

In [32]:
decode_array = create_decode_array(R, M)
decode_array

[array([[[1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 0, 0, 1]],
 
        [[1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1]],
 
        [[1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
     

In [33]:
word = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
w = np.dot(word, G) % 2
w

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [34]:
err1_array = [w ^ x for x in np.eye(len(w), dtype=int)]
err2 = np.zeros_like(w)
err2[0] = err2[1] = 1
err3 = err2.copy()
err3[2] = 1
w_err2 = w ^ err2
w_err3 = w ^ err3
w, w_err2, w_err3

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32),
 array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32),
 array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))

In [35]:
# Декодирование слова без ошибки
decode_word = decoding_word(w, decode_array, G, R, M)
if decode_word is not None:
    print(decode_word)
    print(np.array_equal(word, decode_word))
     

[1 0 0 0 0 0 0 0 0 0 0]
True


In [36]:
# Декодирование всех однократных ошибок
decode_words = [decoding_word(x, decode_array, G, R, M) for x in err1_array]
for x in decode_words:
    print(np.array_equal(x, word))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [37]:
# Декодирование двукратной ошибки (RM(2, 4) может их лишь обнаружить)
decode_word = decoding_word(w_err2, decode_array, G, R, M)
if decode_word is not None:
    print(decode_word)
    print(np.array_equal(word, decode_word))

Допущена 2-я ошибка. Сообщение не может быть восстановлено!


In [38]:
# Декодирование 3-кратной ошибки (RM(2,4) не может их обнаружить)
decode_word = decoding_word(w_err3, decode_array, G, R, M)
if decode_word is not None:
    print(decode_word)
    print(np.array_equal(word, decode_word))

[1 0 0 0 0 1 0 0 0 0 0]
False
